In [6]:
%matplotlib widget
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math

In [7]:
company_symbol = ""
financial_state = ""
base_url = "https://fr.finance.yahoo.com/quote"

# Example: https://fr.finance.yahoo.com/quote/AAPL/balance-sheet?p=AAPL

In [8]:
# Request to yahoo finance to get all the company's financial statements
def get_financial_statement(company_symbol, financial_state):
    url = f"{base_url}/{company_symbol}/{financial_state}?p={company_symbol}"
    with requests.session():
        header = {'Connection': 'keep-alive',
                   'Expires': '-1',
                   'Upgrade-Insecure-Requests': '1',
                   'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) \
                   AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36'
                   }
        page = requests.get(url, headers=header)
        soup = BeautifulSoup(page.content, 'html.parser')
        table = soup.find_all('div', class_="M(0) Whs(n) BdEnd Bdc($seperatorColor) D(itb)")
        return table


company_symbol = "OR.PA"
financial_state = "balance-sheet"
raw_table = get_financial_statement(company_symbol, financial_state)
raw_table

[<div class="M(0) Whs(n) BdEnd Bdc($seperatorColor) D(itb)"><div class="D(tbhg)"><div class="D(tbr) C($primaryColor)"><div class="D(ib) Fw(b) Ta(start) Px(15px)--mv2 Px(10px) W(247px)--mv2 W(222px) Bxz(bb) Bdendw(1px) Bdstartw(1px) Bdbw(1px) Bdends(s) Bdstarts(s) Bdbs(s) Bdc($seperatorColor) Py(6px) Pos(st) Start(0) Bgc($lv2BgColor)"><span>Détails</span><div class="W(3px) Pos(a) Start(100%) T(0) H(100%) Bg($pfColumnFakeShadowGradient) Pe(n) Pend(5px)"></div></div><div class="Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(100px)--pnclg D(ib) Fw(b)"><span>31/12/2022</span></div><div class="Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(100px)--pnclg D(ib) Fw(b) Bgc($lv1BgColor)"><span>31/12/2021</span></div><div class="Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(100px)--pnclg D(ib) Fw(b)"><span>31/12/2020</span></div><div class="Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(100px)--pnclg D(ib) Fw(b) Bgc($lv1BgColor)"><span>3

In [9]:
def get_columns(raw_table):
    soup = BeautifulSoup(str(raw_table[0]), "html.parser")
    raw_table_headers = soup.find("div", class_="D(tbhg)").find_all("span")
    
    headers = [i.text for i in raw_table_headers]
    return headers[1:]


df_headers = get_columns(raw_table)
df_headers

['31/12/2022', '31/12/2021', '31/12/2020', '31/12/2019']

In [10]:
def get_indexes(raw_table):
    indexes = []
    soup = BeautifulSoup(str(raw_table[0]), "html.parser")
    first_row = soup.find("div", class_="D(tbhg)")
    first_index = [span.text for span in first_row.find_all("span")][0]
    raw_tables = soup.find_all("div", class_="D(tbrg)")
    lines = [row.find_all("div", "D(tbr) fi-row Bgc($hoverBgColor):h") for row in raw_tables]
    lines = [row.find_all("span") for row in lines[0] if row.find_all("div",)]
    lines = [row[0].text for row in lines]
    
    print(lines)
    indexes.append(first_index)
    return indexes

df_indexes = get_indexes(raw_table)
df_indexes

['Actifs', 'Actif à court terme', 'Trésorerie', 'Espèces et quasi-espèces', 'Autres investissements à court terme', 'Trésorerie totale', 'Créances nettes', 'Inventaire', 'Autre actif à court terme', 'Total des actifs à court terme', 'Actifs non circulants', 'Terrains, usines et équipements', 'Investissements bruts en terrains, usines et équipements', 'Dépréciation cumulée', 'Investissements nets en terrains, usines et équipements', 'Clientèle', 'Biens incorporels', 'Autres actifs à long terme', 'Total des actifs non circulants', 'Total des actifs', 'Passif et capitaux propres', 'Passif', 'Passif à court terme', 'Dette courante', 'Créances', 'Autre passif à court terme', 'Total des passifs à court terme', 'Passifs non à court terme', 'Dette à long terme', 'Passif d’impôts différé', 'Autres passifs à long terme', 'Total des passifs non à court terme', 'Passifs totaux', 'Capitaux propres des détenteurs', 'Action ordinaire', 'Bénéfices non répartis', 'Total des capitaux propres', 'Passif t

['Détails']

In [11]:
def isStringNumber(string):
    try:
        string = int(string)
        return True
    except:
        return False

string = '-6834400'
isStringNumber(string)

True

In [12]:

def get_values(raw_table):
    res = {}
    soup = BeautifulSoup(str(raw_table[0]), "html.parser")
    cells = soup.find_all("div", class_="D(tbc)")
    # [print(cell.text) for cell in cells]
    for i in range(0, len(cells), len(df_headers) + 1):
        res[cells[i].text.replace('\u202f', '')] = [cell.text.replace('\u202f', '') for cell in cells[i+1:i+len(df_headers) + 1]]
    # for cell in cells:
        
    #     cell = cell.text.replace('\u202f', '')
        # if cell != '' and not isStringNumber(cell) and cell != '-':
        #     category = cell
        #     res[category] = []
        # elif cell != '' and isStringNumber(cell):
        #     res[category].append(int(cell))
        # elif cell == '-':
        #     res[category].append('-')
        # else:
        #     res[category].append('-')
            
        # if cell.text == '':
        #     print('-')
        # else:
        #     print(cell.text)
    
    # [print(span) for span in spans]
    # print(spans[0])
    # for row in spans:
    #     pass
    # for row in spans:
        # print(row)
        # for index, elem in enumerate(row):
        #     elem = elem.text.replace('\u202f', '')
        #     if index == 0:
        #         res[elem] = []
        #     print(elem)
        #     if elem.isdigit():
        #         # append to the last index
        #         res[list(res.keys())[-1]].append(int(elem))
                

    return res

df_values = get_values(raw_table)
df_values

{'Actifs': ['', '', '', ''],
 'Actif à court terme': ['', '', '', ''],
 'Trésorerie': ['', '', '', ''],
 'Espèces et quasi-espèces': ['2617700', '2713800', '6405900', '5286000'],
 'Autres investissements à court terme': ['23100', '4800', '6700', '23300'],
 'Trésorerie totale': ['2640800', '2718600', '6412600', '5309300'],
 'Créances nettes': ['4755500', '4021000', '3511300', '4086700'],
 'Inventaire': ['4079400', '3166900', '2675800', '2920800'],
 'Autre actif à court terme': ['690200', '754000', '451800', '420500'],
 'Total des actifs à court terme': ['14049600',
  '12075800',
  '14560100',
  '13916500'],
 'Actifs non circulants': ['', '', '', ''],
 'Terrains, usines et équipements': ['', '', '', ''],
 'Investissements bruts en terrains, usines et équipements': ['11798900',
  '11528500',
  '11915400',
  '13766500'],
 'Dépréciation cumulée': ['-6834400', '-6754800', '-7164900', '-8230000'],
 'Investissements nets en terrains, usines et équipements': ['4964500',
  '4773700',
  '4750500'

In [13]:
df = pd.DataFrame(df_values)
df = df.transpose()
df.columns = df_headers
df = df.rename_axis('Details')
df.replace({ '': math.nan })

,31/12/2022,31/12/2021,31/12/2020,31/12/2019
Details,,,,
Actifs,NaN,NaN,NaN,NaN
Actif à court terme,NaN,NaN,NaN,NaN
Trésorerie,NaN,NaN,NaN,NaN
Espèces et quasi-espèces,2617700,2713800,6405900,5286000
Autres investissements à court terme,23100,4800,6700,23300
Trésorerie totale,2640800,2718600,6412600,5309300
Créances nettes,4755500,4021000,3511300,4086700
Inventaire,4079400,3166900,2675800,2920800
Autre actif à court terme,690200,754000,451800,420500


In [14]:
df

,31/12/2022,31/12/2021,31/12/2020,31/12/2019
Details,,,,
Actifs,,,,
Actif à court terme,,,,
Trésorerie,,,,
Espèces et quasi-espèces,2617700,2713800,6405900,5286000
Autres investissements à court terme,23100,4800,6700,23300
Trésorerie totale,2640800,2718600,6412600,5309300
Créances nettes,4755500,4021000,3511300,4086700
Inventaire,4079400,3166900,2675800,2920800
Autre actif à court terme,690200,754000,451800,420500


In [5]:
df.to_csv(f'{company_symbol}_{financial_state}.csv')

NameError: name 'df' is not defined

In [100]:
financial_state

''

In [15]:
url = "https://fr.finance.yahoo.com/quote/%5EFCHI/components?p=%5EFCHI"
with requests.session():
    header = {'Connection': 'keep-alive',
            'Expires': '-1',
            'Upgrade-Insecure-Requests': '1',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) \
            AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36'
            }
    page = requests.get(url, headers=header)
    soup = BeautifulSoup(page.content, 'html.parser')
    table = soup.find_all('tr', class_="BdT Bdc($seperatorColor) Ta(end) Fz(s)")

In [16]:
print(table)

[<tr class="BdT Bdc($seperatorColor) Ta(end) Fz(s)"><td class="Py(10px) Ta(start) Pend(10px)"><a class="C($linkColor) Cur(p) Td(n) Fw(500)" href="/quote/OR.PA?p=OR.PA" title="OR.PA">OR.PA</a></td><td class="Py(10px) Ta(start) Pend(10px)">L'Oreal SA</td><td class="Py(10px) Pstart(10px)">406,40</td><td class="Py(10px) Pstart(10px)">0,10</td><td class="Py(10px) Pstart(10px) C($positiveColor)">+<!-- -->0,02 %</td><td class="Py(10px) Pstart(10px)">241 006</td></tr>, <tr class="BdT Bdc($seperatorColor) Ta(end) Fz(s)"><td class="Py(10px) Ta(start) Pend(10px)"><a class="C($linkColor) Cur(p) Td(n) Fw(500)" href="/quote/VIV.PA?p=VIV.PA" title="VIV.PA">VIV.PA</a></td><td class="Py(10px) Ta(start) Pend(10px)">Vivendi SE</td><td class="Py(10px) Pstart(10px)">8,27</td><td class="Py(10px) Pstart(10px)">-0,00</td><td class="Py(10px) Pstart(10px) C($negativeColor)">-0,05 %</td><td class="Py(10px) Pstart(10px)">1 340 583</td></tr>, <tr class="BdT Bdc($seperatorColor) Ta(end) Fz(s)"><td class="Py(10px) T

In [33]:
for row in table:
    print(row.find_all('td', class_="Py(10px) Ta(start) Pend(10px)")[0].text)

OR.PA
VIV.PA
GLE.PA
SW.PA
HO.PA
BN.PA
CAP.PA
RI.PA
SGO.PA
EN.PA
SU.PA
AC.PA
ENGI.PA
AIR.PA
ML.PA
LR.PA
AI.PA
CA.PA
SAN.PA
MC.PA
DG.PA
VIE.PA
ORA.PA
ACA.PA
BNP.PA
WLN.PA
ATO.PA
KER.PA
UG.PA
FP.PA
